# Code to access and store Shape Files for Buncombe County

## Notes
09-06-18: on collier machine this works with anaconda geospatial env (source activate geospatial)



In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
from urllib2 import urlopen
from zipfile import ZipFile
from io import BytesIO
import shapefile
from shapely.geometry import shape
from json import dumps
import osr
import sys
import os

### grab the shape files off the web

In [2]:
zipped_shp_url = 'http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_37_bg_500k.zip'

zipfile = ZipFile(BytesIO(urlopen(zipped_shp_url).read()))
filenames = [y for y in sorted(zipfile.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
dbf, prj, shp, shx = [BytesIO(zipfile.read(filename)) for filename in filenames]
r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)
    
attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]  
for row in r.shapeRecords():  
    geometry.append(shape(row.shape.__geo_interface__))  
    attributes.append(dict(zip(field_names, row.record)))  

In [3]:
proj4_string = osr.SpatialReference(prj.read()).ExportToProj4()
'this projection is {0}'.format(proj4_string)

'this projection is +proj=longlat +ellps=GRS80 +no_defs '

### make it into a geopandas dataframe

In [12]:
all_nc_tract_geo = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = proj4_string)

In [13]:
buncombe_tract_geo = all_nc_tract_geo[all_nc_tract_geo.COUNTYFP=='021']
buncombe_tract_geo.index = buncombe_tract_geo['GEOID']
buncombe_tract_geo.head()

,AFFGEOID,ALAND,AWATER,BLKGRPCE,COUNTYFP,GEOID,LSAD,NAME,STATEFP,TRACTCE,geometry
GEOID,,,,,,,,,,,
370210003002,1500000US370210003002,461349,0,2,021,370210003002,BG,2,37,000300,"POLYGON ((-82.56777 35.606625, -82.564109 35.6..."
370210019001,1500000US370210019001,2416732,17389,1,021,370210019001,BG,1,37,001900,"POLYGON ((-82.502804 35.58734, -82.501977 35.5..."
370210022042,1500000US370210022042,2191343,4393,2,021,370210022042,BG,2,37,002204,"POLYGON ((-82.536917 35.509338, -82.537559 35...."
370210005003,1500000US370210005003,1999352,0,3,021,370210005003,BG,3,37,000500,"POLYGON ((-82.554647 35.623198, -82.5539359999..."
370210012003,1500000US370210012003,606041,0,3,021,370210012003,BG,3,37,001200,"POLYGON ((-82.60565199999999 35.573349, -82.60..."


### Save the result
NOTE: we must save this as a geospatial file!

In [14]:
#make sure directory exists to store shapefiles
if not os.path.isdir('shape_file_dir'):
   !mkdir shape_file_dir 

In [15]:
buncombe_tract_geo.to_file(driver = 'ESRI Shapefile', filename= "./shape_file_dir/buncombe_bg.shp")

### make geojson version of shapefiles

In [16]:
# read the shapefile
reader = shapefile.Reader("./shape_file_dir/buncombe_bg.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 
 

In [19]:
#make sure directory exists to store shapefiles
if not os.path.isdir('geojson_file_dir'):
   !mkdir geojson_file_dir 

In [20]:
# write the GeoJSON file
geojson = open("geojson_file_dir/buncombe_bg.json", "w")
geojson.write(dumps({"type": "FeatureCollection",\
    "features": buffer}, indent=2) + "\n")
geojson.close()